# NASA - CLEAN

## Setup 

In [344]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sys, os, yaml

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', None)

sns.set_style("darkgrid")

DATASET = "NASA"
COLAB = 'google.colab' in sys.modules

from IPython.display import display, Markdown
from pprint import pprint 

DEBUG = True

In [345]:
COLAB = 'google.colab' in sys.modules

if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
  if not os.path.isdir(ROOT): os.makedirs(ROOT)
else:
  ROOT = "./"

def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

## Load Dataset

In [346]:
filename = f"{ROOT}/orig/data.csv"

if os.path.isfile(filename):
    print("Using local copy ... ")
else:
    print("Downloading ... ")
    df = pd.read_csv("https://setu-datamining2.github.io/live/topics/21-Assignments/01-NASA_Software_Defect_Datasets/files/pc2.csv")
    df.to_csv(filename, index=False)
    
df = pd.read_csv(filename)
print(df.shape)
df.head()

Using local copy ... 
(5589, 37)


,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,DESIGN_COMPLEXITY,DESIGN_DENSITY,EDGE_COUNT,ESSENTIAL_COMPLEXITY,ESSENTIAL_DENSITY,LOC_EXECUTABLE,PARAMETER_COUNT,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_EFFORT,HALSTEAD_ERROR_EST,HALSTEAD_LENGTH,HALSTEAD_LEVEL,HALSTEAD_PROG_TIME,HALSTEAD_VOLUME,MAINTENANCE_SEVERITY,MODIFIED_CONDITION_COUNT,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defects
0,1.0,0.0,0.0,0.0,0.0,1.0,1.00,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,5.33,1.50,12.00,0.00,4.0,0.67,0.67,8.00,1.0,0.0,0.0,2.0,0.50,1.0,3.0,1.0,3.0,2.0,0.00,0.0,False
1,1.0,1.0,0.0,0.0,0.0,1.0,1.00,0.0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,1.0,0.0,0.0,3.0,1.00,1.0,0.0,1.0,0.0,1.0,0.00,0.0,False
2,1.0,4.0,7.0,24.0,0.0,1.0,0.13,0.0,0.0,1.0,1.0,6.0,1.0,0.0,1.0,0.0,17.88,7.43,986.77,0.04,34.0,0.13,54.82,132.83,1.0,0.0,0.0,7.0,0.03,13.0,21.0,7.0,8.0,34.0,96.88,8.0,False
3,1.0,1.0,11.0,3.0,0.0,1.0,0.08,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,42.62,7.81,2598.31,0.11,77.0,0.13,144.35,332.79,1.0,0.0,0.0,3.0,0.06,29.0,48.0,13.0,7.0,17.0,93.33,12.0,False
4,1.0,1.0,0.0,0.0,0.0,1.0,1.00,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,3.0,33.44,0.63,13.06,0.01,9.0,1.60,0.73,20.90,1.0,0.0,0.0,3.0,0.33,5.0,4.0,4.0,1.0,3.0,0.00,1.0,False


## Table 1 - Summary;

In [347]:
data = []
for message, expected, observed in [
    ("Number of cases", 5589, df.shape[0]),
    ("Number of features", 37, df.shape[1]),
]:
    data.append([message,expected,observed])
    
df_result = pd.DataFrame(data, columns=['Message','Expected','Observed'])
df_result.head()

,Message,Expected,Observed
0,Number of cases,5589,5589
1,Number of features,37,37


## Table 3 - by Features

DataFrame that will flag the columns with error by Data Quality Category

In [348]:
feature_list=df.columns
zero_data = np.zeros(shape=(13,len(feature_list)))
df_error = pd.DataFrame(zero_data, columns=feature_list)

In [349]:
def set_error_flag (ft,names):
        df_error.loc[ft,names] = 1

### A - Identical features

In [350]:
def table_3_A (df):
    df_tmp = df.T.duplicated()
    df_tmp = df_tmp[df_tmp == True]
    p_A_count = df_tmp.shape[0]
    p_A_columns = set(df_tmp.index)
    
    if p_A_count:
        set_error_flag(0,list(p_A_columns))
        
    result = list(df_error.sum(axis=1))
    return result[0],p_A_columns

table_3_A_observed, p_A_columns = table_3_A (df)
print(table_3_A_observed, p_A_columns)

0.0 set()


### B - Constant features

In [351]:
def table_3_B (df):
    df_tmp = df.nunique()
    df_tmp = df_tmp[df_tmp<=1]
    p_B_count = df_tmp.shape[0]
    p_B_columns = set(df_tmp.index)
    
    if p_B_count:
        set_error_flag(1,list(p_B_columns))
        
    result = list(df_error.sum(axis=1))
    return result[1], p_B_columns

table_3_B_observed, p_B_columns = table_3_B (df)
print(table_3_B_observed, p_B_columns)

0.0 set()


### C - Features with missing values

In [352]:
def table_3_C (df):
    df_tmp = df.isna().sum()
    df_tmp = df_tmp[df_tmp>0]
    p_C_count = df_tmp.shape[0]
    p_C_columns = set(df_tmp.index)
    
    if p_C_count:
        set_error_flag(2,list(p_C_columns))

    result = list(df_error.sum(axis=1))
    return result[2]

table_3_C_observed = table_3_C (df)

### D - Features with conflicting values

###### (13) HALSTEAD VOLUME = (NUM OPERATORS+NUM OPERANDS)*log2(NUM UNIQUE OPERATORS+NUM UNIQUE OPERANDS)

 X = (A + B) * log2(C + D) ----> (C + D) = 2 ** (X / (A + B))

In [353]:
(((df.HALSTEAD_VOLUME)==(df.NUM_OPERATORS + df.NUM_OPERANDS) * np.log2(df.NUM_UNIQUE_OPERATORS + df.NUM_UNIQUE_OPERANDS)) == ((df.NUM_UNIQUE_OPERATORS + df.NUM_UNIQUE_OPERANDS) == 2**(df.HALSTEAD_VOLUME / (df.NUM_OPERATORS + df.NUM_OPERANDS)))).sum()

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


5589

In [354]:
a = (df.NUM_UNIQUE_OPERATORS + df.NUM_UNIQUE_OPERANDS)

In [355]:
b = (df.HALSTEAD_VOLUME / (df.NUM_OPERATORS + df.NUM_OPERANDS))
b = b.replace(np.nan,0)
b = 2 ** b

In [356]:
integrity_13 = (~np.isclose(a,b, atol=1))

###### (14) HALSTEAD LEVEL = (2/NUM UNIQUE OPERATORS)*(NUM UNIQUE OPERANDS/NUM OPERANDS)

In [357]:
a = df.HALSTEAD_LEVEL

In [358]:
b = df.NUM_UNIQUE_OPERANDS/df.NUM_OPERANDS
b = b.replace(np.nan,0)
b = b * (2/df.NUM_UNIQUE_OPERATORS)

In [359]:
integrity_14 = (~np.isclose(a,b, atol=1))

###### (15) HALSTEAD DIFFICULTY = (NUM UNIQUE OPERATORS/2)*(NUM OPERANDS/NUM UNIQUE OPERANDS)

In [360]:
a = df.HALSTEAD_DIFFICULTY

In [361]:
b = (df.NUM_OPERANDS/df.NUM_UNIQUE_OPERANDS)
b = b.replace(np.nan,0)
b = b*(df.NUM_UNIQUE_OPERATORS/2)

In [362]:
integrity_15 = (~np.isclose(a,b, atol=1))

###### (16) HALSTEAD CONTENT = HALSTEAD VOLUME/HALSTEAD DIFFICULTY


In [363]:
a = df.HALSTEAD_CONTENT

In [364]:
b = df.HALSTEAD_VOLUME / df.HALSTEAD_DIFFICULTY
b = b.replace(np.nan,0)

In [365]:
integrity_16 = (~np.isclose(a,b, atol=1))

###### (17) HALSTEAD EFFORT = HALSTEAD VOLUME*HALSTEAD DIFFICULTY

In [366]:
a = df.HALSTEAD_EFFORT/df.HALSTEAD_VOLUME
a = a.replace(np.nan,0)

In [367]:
b = df.HALSTEAD_DIFFICULTY

In [368]:
integrity_17 = (~np.isclose(a,b, atol=1))

#### Checks

In [369]:
integrity_checks = {
    
    (df.NUMBER_OF_LINES < df.LOC_TOTAL).sum()                                   : ['NUMBER_OF_LINES','LOC_TOTAL'],                          # (1) NUMBER OF LINES ≥ LOC TOTAL
    (df.NUMBER_OF_LINES < df.LOC_CODE_AND_COMMENT).sum()                        : ['NUMBER_OF_LINES','LOC_CODE_AND_COMMENT'],               # (3) NUMBER OF LINES ≥ LOC CODE AND COMMENT
    (df.NUMBER_OF_LINES < df.LOC_COMMENTS).sum()                                : ['NUMBER_OF_LINES','LOC_COMMENTS'],                       # (4) NUMBER OF LINES ≥ LOC COMMENTS
    (df.NUMBER_OF_LINES < df.LOC_EXECUTABLE).sum()                              : ['NUMBER_OF_LINES','LOC_EXECUTABLE'],                     # (5) NUMBER OF LINES ≥ LOC EXECUTABLE
    (df.LOC_TOTAL < df.LOC_EXECUTABLE).sum()                                    : ['LOC_EXECUTABLE','LOC_TOTAL'],                           # (6) LOC TOTAL ≥ LOC EXECUTABLE
    (df.LOC_TOTAL < df.LOC_CODE_AND_COMMENT).sum()                              : ['LOC_CODE_AND_COMMENT','LOC_TOTAL'],                     # (7) LOC TOTAL ≥ LOC CODE AND COMMENT
    (df.NUM_OPERANDS < df.NUM_UNIQUE_OPERANDS).sum()                            : ['NUM_OPERANDS','NUM_UNIQUE_OPERANDS'],                   # (8) NUM OPERANDS ≥ NUM UNIQUE OPERANDS
    (df.NUM_OPERATORS < df.NUM_UNIQUE_OPERATORS).sum()                          : ['NUM_OPERATORS','NUM_UNIQUE_OPERATORS'],                 # (9) NUM OPERATORS ≥ NUM UNIQUE OPERATORS
    (df.HALSTEAD_LENGTH != (df.NUM_OPERATORS + df.NUM_OPERANDS)).sum()          : ['HALSTEAD_LENGTH','NUM_OPERATORS','NUM_OPERANDS'],       # (10) HALSTEAD LENGTH = NUM OPERATORS + NUM OPERANDS
    (df.CYCLOMATIC_COMPLEXITY > (df.NUM_OPERATORS)+1).sum()                     : ['CYCLOMATIC_COMPLEXITY','NUM_OPERATORS'],                # (11) CYCLOMATIC COMPLEXITY ≤ NUM OPERATORS+1
    (df.CALL_PAIRS > df.NUM_OPERATORS).sum()                                    : ['CALL_PAIRS','NUM_OPERATORS'],                           # (12) CALL PAIRS ≤ NUM OPERATORS
    integrity_13.sum()                                                          : ['HALSTEAD_VOLUME','NUM_OPERATORS','NUM_OPERANDS','NUM_UNIQUE_OPERATORS','NUM_UNIQUE_OPERANDS'],
    integrity_14.sum()                                                          : ['HALSTEAD_LEVEL','NUM_UNIQUE_OPERATORS','NUM_OPERANDS','NUM_UNIQUE_OPERANDS'],
    integrity_15.sum()                                                          : ['HALSTEAD_DIFFICULTY','NUM_UNIQUE_OPERATORS','NUM_OPERANDS','NUM_UNIQUE_OPERANDS'],
    integrity_16.sum()                                                          : ['HALSTEAD_CONTENT','HALSTEAD_VOLUME','HALSTEAD_DIFFICULTY'],
    integrity_17.sum()                                                          : ['HALSTEAD_EFFORT','HALSTEAD_DIFFICULTY','HALSTEAD_VOLUME'],
    (~np.isclose((df.HALSTEAD_PROG_TIME*18),df.HALSTEAD_EFFORT,atol=1)).sum()   : ['HALSTEAD_PROG_TIME','HALSTEAD_EFFORT'], 

}

In [370]:
def table_3_D (df):
    for v,k in integrity_checks.items():
        if (v>0) :
            set_error_flag(3,k)
    result = list(df_error.sum(axis=1))
    return result[3]

table_3_D_observed = table_3_D(df)

### E - Featuring with implausible values

* LOC TOTAL = 0

In [371]:
loc_count = ((df.LOC_TOTAL==0).sum()>0).sum()

* value of any attribute is < 0

In [372]:
df_tmp = (df<0).sum()
df_tmp = df_tmp[(df_tmp>0)]
negative_count = df_tmp.shape[0]
negative_columns = set(df_tmp.index)

* any count is a non-integer

In [373]:
counts_columns = list([c for c in list(df.columns) if "COUNT" in c])

df_tmp_counts = df[counts_columns]
df_tmp_integers = df_tmp_counts.select_dtypes(['int','float'])

non_integer_count = df_tmp_counts.shape[1] - df_tmp_integers.shape[1]
non_integer_columns = counts_columns

for c in df_tmp_integers.columns:
    if c in counts_columns:
        non_integer_columns.remove(c)

In [374]:
def table_3_e_aux(count,names):
    if count:
        set_error_flag(4,names) 
    
def table_3_E (df):
    
    table_3_e_aux(negative_count, list(negative_columns))
    
    table_3_e_aux(loc_count,'LOC_TOTAL')
    
    table_3_e_aux(non_integer_count, non_integer_columns)   
       
    result = list(df_error.sum(axis=1))
    return result[4]

table_3_E_observed = table_3_E (df)

### F - Total Problem Features

In [375]:
def table_3_F (df):
    result = df_error.sum().sum()
    return result

table_3_F_observed = table_3_F (df)
print(table_3_F_observed)

10.0


## Table 4 - by Cases (rows)

### G - Identical cases

In [376]:
def table_4_G (df):
    
    df_tmp = df.duplicated(keep=False)
    df_tmp = df_tmp[df_tmp]
    p_G_count = df_tmp.shape[0]
    p_G_rows_all = set(df_tmp.index)
    
    df_tmp2 = df.duplicated(keep='first')
    df_tmp2 = df_tmp2[df_tmp2]
    p_G_rows_except_first = set(df_tmp2.index)
    
    return p_G_count, p_G_rows_all, p_G_rows_except_first

table_4_G_observed, p_G_rows_all, p_G_rows_except_first = table_4_G(df)
print("Duplicates : ",table_4_G_observed,p_G_rows_all,"\nDuplicates to remove : ",len(p_G_rows_except_first), p_G_rows_except_first)

Duplicates :  4621 {0, 1, 4, 5, 6, 7, 8, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 27, 29, 30, 33, 34, 35, 37, 38, 39, 40, 41, 42, 45, 46, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 65, 66, 69, 70, 71, 72, 73, 75, 76, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 106, 107, 108, 109, 110, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 144, 145, 148, 150, 151, 152, 153, 154, 156, 157, 159, 160, 162, 163, 164, 165, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 203, 205, 206, 207, 208, 209, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 225, 226, 227, 228, 229, 230, 231, 232, 233, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 249, 250, 251, 254, 256, 257, 258, 260, 261, 262, 263, 264, 265,

### H - Inconsistent cases

In [377]:
from itertools import combinations

def inconsistent_row(df,target='defects'):
    columns = [c for c in df.columns if c != target]

    df_tmp = df[df.duplicated(subset=columns, keep=False)]

    inconsistent = []
    for _, group in df_tmp.groupby(columns):
        if len(group[target].unique()) > 1:
            group_index = group.index.tolist()
            for i, j in combinations(group_index, 2):
                if df.loc[i, target] != df.loc[j, target]:
                    inconsistent.append(i)
                    inconsistent.append(j)
                    
    inconsistent = set(inconsistent)
    return len(inconsistent), inconsistent

In [378]:
def table_4_H (df):
    return inconsistent_row(df)

table_4_H_observed,p_H_rows = table_4_H(df)
print(table_4_H_observed,p_H_rows)

100 {5121, 1031, 8, 521, 4107, 4108, 5134, 3604, 4631, 1057, 37, 2602, 2094, 3645, 3134, 4159, 2627, 1615, 1627, 2657, 4204, 2669, 4206, 5229, 113, 2673, 2163, 116, 1140, 3705, 2169, 2697, 2700, 1677, 3728, 4775, 687, 3760, 5297, 4788, 2748, 1217, 2241, 4817, 723, 2270, 2271, 3297, 2789, 230, 236, 1262, 2804, 3834, 3323, 1794, 4354, 776, 4364, 3856, 1809, 1303, 3357, 4394, 1835, 3372, 4917, 826, 1339, 319, 1349, 1351, 5447, 1868, 3928, 2921, 875, 1387, 367, 370, 887, 4480, 3981, 2965, 1950, 418, 2466, 5552, 2482, 1461, 3515, 968, 1500, 2525, 5090, 4580, 4071, 3058, 2036, 5109}


### I - Cases with missing 

In [379]:
def table_4_I (df):
    df_tmp = df.isna().sum(axis=1)
    df_tmp = df_tmp[df_tmp>0]
    p_I_count = df_tmp.shape[0]
    p_I_rows = set(df_tmp.index)
    return p_I_count, p_I_rows

table_4_I_observed,p_I_rows = table_4_I(df)
print(table_4_I_observed,p_I_rows)

0 set()


### J - Cases with conflicting feature values

In [380]:
integrity_checks_rows = [
    (df.NUMBER_OF_LINES < df.LOC_TOTAL),                                    # (1) NUMBER OF LINES ≥ LOC TOTAL
    (df.NUMBER_OF_LINES < df.LOC_CODE_AND_COMMENT),                         # (3) NUMBER OF LINES ≥ LOC CODE AND COMMENT
    (df.NUMBER_OF_LINES < df.LOC_COMMENTS),                                 # (4) NUMBER OF LINES ≥ LOC COMMENTS
    (df.NUMBER_OF_LINES < df.LOC_EXECUTABLE),                               # (5) NUMBER OF LINES ≥ LOC EXECUTABLE
    (df.LOC_TOTAL < df.LOC_EXECUTABLE),                                     # (6) LOC TOTAL ≥ LOC EXECUTABLE
    (df.LOC_TOTAL < df.LOC_CODE_AND_COMMENT),                               # (7) LOC TOTAL ≥ LOC CODE AND COMMENT
    (df.NUM_OPERANDS < df.NUM_UNIQUE_OPERANDS),                             # (8) NUM OPERANDS ≥ NUM UNIQUE OPERANDS
    (df.NUM_OPERATORS < df.NUM_UNIQUE_OPERATORS),                           # (9) NUM OPERATORS ≥ NUM UNIQUE OPERATORS
    (df.HALSTEAD_LENGTH != (df.NUM_OPERATORS + df.NUM_OPERANDS)),           # (10) HALSTEAD LENGTH = NUM OPERATORS + NUM OPERANDS
    (df.CYCLOMATIC_COMPLEXITY > (df.NUM_OPERATORS)+1),                      # (11) CYCLOMATIC COMPLEXITY ≤ NUM OPERATORS+1
    (df.CALL_PAIRS > df.NUM_OPERATORS),                                     # (12) CALL PAIRS ≤ NUM OPERATORS
    integrity_13,                                                           # (13) HALSTEAD VOLUME = (NUM OPERATORS+NUM OPERANDS)*log2(NUM UNIQUE OPERATORS+NUM UNIQUE OPERANDS)
    integrity_14,                                                           # (14) HALSTEAD LEVEL = (2/NUM UNIQUE OPERATORS)*(NUM UNIQUE OPERANDS/NUM OPERANDS)
    integrity_15,                                                           # (15) HALSTEAD DIFFICULTY = (NUM UNIQUE OPERATORS/2)*(NUM OPERANDS/NUM UNIQUE OPERANDS)
    integrity_16,                                                           # (16) HALSTEAD CONTENT = HALSTEAD VOLUME/HALSTEAD DIFFICULTY
    integrity_17,                                                           # (17) HALSTEAD EFFORT = HALSTEAD VOLUME*HALSTEAD DIFFICULTY
    (~np.isclose((df.HALSTEAD_PROG_TIME*18),df.HALSTEAD_EFFORT,atol=1)),    # (18) HALSTEAD PROG TIME = HALSTEAD EFFORT/18 
]

In [381]:
def table_4_J (df):
    index = sum(integrity_checks_rows)>0
    p_J_rows = set([r for r in range(len(index)) if index[r]])  
    return len(p_J_rows), p_J_rows

table_4_J_observed,p_J_rows = table_4_J(df)
print(table_4_J_observed,p_J_rows)

271 {1536, 1, 1540, 5127, 3592, 1033, 4105, 2059, 5139, 1046, 5144, 4633, 538, 5149, 1056, 1569, 2080, 3104, 4128, 3621, 5153, 39, 551, 2089, 5156, 5159, 5163, 2605, 5166, 4658, 1075, 1079, 4663, 2109, 1087, 4162, 4674, 1611, 2635, 5200, 4689, 595, 2644, 597, 1621, 3160, 4697, 3676, 2142, 5216, 2145, 3170, 4194, 2660, 2662, 4711, 1640, 5222, 4203, 2160, 4212, 3701, 2167, 3703, 1145, 2170, 1659, 3195, 4221, 4222, 127, 3711, 4736, 4739, 2695, 3722, 2701, 1167, 2704, 3727, 1683, 1172, 4244, 664, 1688, 2713, 3225, 4253, 672, 1185, 2210, 3748, 4265, 683, 3756, 2221, 5292, 688, 2226, 4786, 2741, 3254, 3765, 5305, 4283, 5311, 2753, 194, 1731, 4805, 3270, 4806, 4808, 1226, 1229, 2254, 3790, 4303, 5331, 3799, 2264, 218, 219, 1245, 734, 4829, 2786, 5351, 232, 2792, 2794, 1260, 4338, 2291, 244, 3829, 2806, 5367, 3832, 1275, 2300, 253, 3835, 767, 3842, 3331, 2314, 5386, 2828, 1805, 4368, 4880, 3346, 4883, 3862, 791, 2328, 4887, 3867, 4379, 1309, 3869, 1823, 800, 3359, 802, 5409, 1828, 4388, 806, 5

### K - Cases with implausible values

* LOC TOTAL = 0

In [382]:
def loc_zero_rows(df):
    index = (df.LOC_TOTAL==0)
    loc_rows = set([r for r in range(len(index)) if index[r]])  
    print("ROWS_LOC_TOTAL = 0 :",len(loc_rows), loc_rows)
    return loc_rows

* value of any attribute is < 0

In [383]:
def negative_rows(df):
    index = (df<0).T.sum()>0
    negative_rows = set([r for r in range(len(index)) if index[r]])  
    print("ROWS with Negative value : ", len(negative_rows), negative_rows)
    return negative_rows

* any count is a non-integer

In [384]:
def non_integer_rows(df):
    counts_columns = list([c for c in list(df.columns) if "COUNT" in c])
    df_tmp_counts = df[counts_columns]
    index = df_tmp_counts.applymap(np.isreal).sum(axis=1)<len(counts_columns)
    non_integer_rows = set([r for r in range(len(index)) if index[r]])  
    print("ROWS with non-integer value on count columns : ",len(non_integer_rows), non_integer_rows)
    return non_integer_rows

In [385]:
def table_4_K (df):
    p_K_rows = loc_zero_rows(df) | negative_rows(df) | non_integer_rows(df)
    return len(p_K_rows), p_K_rows

table_4_K_observed, p_K_rows = table_4_K(df)
print(table_4_K_observed,p_K_rows)

ROWS_LOC_TOTAL = 0 : 1084 {0, 1, 2051, 2052, 5, 2054, 4102, 4103, 2057, 4105, 12, 2065, 18, 4115, 22, 4118, 24, 2072, 4120, 4125, 4127, 4128, 35, 38, 2087, 40, 2089, 2090, 4139, 2092, 45, 4141, 4145, 2098, 2099, 52, 2101, 54, 4147, 59, 2109, 2111, 4162, 4164, 70, 2118, 2119, 4166, 2125, 78, 80, 2128, 2132, 85, 86, 2133, 88, 4180, 4181, 2140, 93, 95, 97, 2145, 2148, 4196, 104, 2154, 4203, 2156, 4208, 4212, 2166, 119, 2167, 4215, 4216, 125, 4221, 127, 4222, 129, 4224, 2180, 133, 2182, 136, 2187, 4236, 4237, 2193, 148, 4244, 2198, 154, 2202, 2209, 2210, 4257, 2212, 2213, 4258, 167, 2215, 2216, 4260, 4264, 4265, 4270, 177, 2225, 2235, 188, 4283, 4286, 4287, 4289, 4290, 4292, 197, 198, 4295, 4297, 203, 206, 2254, 2255, 4303, 4307, 4312, 218, 2266, 2267, 4316, 4318, 4319, 2273, 228, 4325, 2281, 4332, 237, 2285, 4334, 241, 242, 2291, 244, 4338, 4339, 4342, 2298, 2300, 2301, 4348, 2305, 4356, 261, 264, 2312, 2314, 2317, 2318, 272, 4368, 274, 2323, 4373, 282, 4379, 4380, 287, 2339, 4388, 4390, 

### L - Total Problem cases, DS' (I - k)

In [386]:
def table_4_L ():
    p_L_rows = set(p_K_rows | p_J_rows | p_I_rows)
    p_L_count = len(p_L_rows)
    return p_L_count,p_L_rows

table_4_L_observed, p_L_rows = table_4_L()
print(table_4_L_observed,p_L_rows)

1174 {0, 1, 5, 4102, 4103, 4105, 12, 18, 4115, 22, 4118, 24, 4120, 4125, 4127, 4128, 35, 38, 39, 40, 4139, 45, 4141, 4145, 4147, 52, 54, 59, 4162, 4164, 70, 4166, 78, 80, 4180, 85, 86, 4181, 88, 93, 95, 97, 4194, 4196, 104, 4203, 4208, 4212, 119, 4215, 4216, 125, 4221, 127, 4222, 129, 4224, 133, 136, 4236, 4237, 148, 4244, 154, 4253, 4257, 4258, 4260, 167, 4264, 4265, 4270, 177, 4283, 188, 4286, 4287, 4289, 4290, 194, 4292, 197, 198, 4295, 4297, 203, 206, 4303, 4307, 4312, 218, 219, 4316, 4318, 4319, 228, 4325, 232, 4332, 237, 4334, 241, 242, 4338, 244, 4339, 4342, 4348, 253, 4356, 261, 264, 272, 4368, 274, 4373, 282, 4379, 4380, 287, 4388, 4390, 297, 4393, 303, 304, 4404, 4405, 4406, 4409, 4413, 321, 4417, 4423, 328, 4431, 338, 4436, 4439, 346, 4443, 349, 350, 352, 353, 4450, 355, 357, 4453, 361, 363, 364, 366, 4462, 373, 374, 4472, 378, 4479, 388, 4485, 395, 397, 401, 4498, 404, 4502, 411, 412, 4508, 4510, 416, 4512, 4517, 426, 427, 4522, 429, 4524, 433, 4535, 4538, 4539, 4540, 447, 

### M - Total Problem cases, DS'' (G - K)

In [387]:
def table_4_M ():
    p_M_rows = set(p_L_rows | p_G_rows_all | p_H_rows)
    p_M_count = len(p_M_rows)
    return p_M_count,p_M_rows

table_4_M_observed, p_M_rows = table_4_M()
print(table_4_M_observed,p_M_rows)

4658 {0, 1, 4, 5, 6, 7, 8, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 27, 29, 30, 33, 34, 35, 37, 38, 39, 40, 41, 42, 45, 46, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 65, 66, 69, 70, 71, 72, 73, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 106, 107, 108, 109, 110, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142, 144, 145, 148, 150, 151, 152, 153, 154, 156, 157, 159, 160, 162, 163, 164, 165, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 203, 205, 206, 207, 208, 209, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 225, 226, 227, 228, 229, 230, 231, 232, 233, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 249, 250, 251, 253, 254, 256, 257, 258, 260, 261, 262, 263, 264, 265, 267,

In [388]:
def to_remove ():
    p_Total_rows = set (p_K_rows | p_J_rows | p_I_rows | p_G_rows_except_first | p_H_rows)
    p_Total_count = len(p_Total_rows)
    return p_Total_count,p_Total_rows

p_Total_count,p_Total_rows = to_remove()
print(p_Total_count,p_Total_rows)

4299 {0, 1, 5, 8, 12, 18, 22, 24, 35, 37, 38, 39, 40, 41, 45, 48, 49, 52, 54, 59, 62, 65, 69, 70, 71, 73, 76, 78, 80, 81, 83, 84, 85, 86, 88, 91, 93, 94, 95, 97, 100, 104, 105, 113, 116, 119, 121, 125, 127, 128, 129, 130, 133, 135, 136, 139, 140, 141, 145, 148, 151, 152, 154, 156, 162, 163, 167, 170, 172, 177, 181, 183, 187, 188, 189, 193, 194, 195, 197, 198, 199, 200, 203, 205, 206, 208, 212, 213, 215, 217, 218, 219, 221, 222, 225, 227, 228, 230, 231, 232, 233, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 253, 254, 256, 260, 261, 263, 264, 270, 271, 272, 273, 274, 277, 282, 283, 285, 286, 287, 288, 289, 290, 294, 295, 296, 297, 299, 301, 302, 303, 304, 305, 307, 309, 310, 311, 314, 315, 317, 318, 319, 321, 324, 327, 328, 330, 331, 335, 336, 337, 338, 339, 341, 342, 346, 347, 349, 350, 351, 352, 353, 354, 355, 356, 357, 359, 360, 361, 363, 364, 366, 367, 369, 370, 371, 372, 373, 374, 376, 377, 378, 380, 382, 385, 387, 388, 392, 394, 395, 396, 397, 401, 402, 404, 405, 406, 410, 411

## Results

In [410]:
def highlight_col(x):
    unmatch_color = 'background-color: #FFFFE0'
    label_color = ['background-color: #8DCFEC','background-color: #B8E2F4']
    
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    index = list(x.query("Expected != Observed").index)
    
    for k in range (df1.shape[0]):
        df1.iloc[k,0:3] = label_color[~k%2]
    
    for k in index:
        df1.iloc[k,1:3] = unmatch_color
    
    
    return df1 

In [413]:
data = []
for message, expected, observed in [
    ("Identical features", 0, table_3_A_observed),
    ("Constant features", 0, table_3_B_observed),
    ("Features with missing values", 0, table_3_C_observed),
    ("Features with conflicting values", 2, table_3_D_observed),
    ("Features with implausible values", 1, table_3_E_observed),
    ("Total Problem Features", 3, table_3_F_observed),
    ("Identical cases", 4621, table_4_G_observed),
    ("Inconsistent cases", 100, table_4_H_observed),
    ("Cases with missing values", 0, table_4_I_observed),
    ("Cases with conflicting feature values", 129, table_4_J_observed),
    ("Cases with implausible values", 1084, table_4_K_observed),
    ("Total Problem cases, DS'",1163,table_4_L_observed),
    ("Total Problem cases, DS''",4297,table_4_M_observed),
    ("Total cases to remove",4297,p_Total_count),
]:
    data.append([message,expected,observed])
    
df_result = pd.DataFrame(data, columns=['Message','Expected','Observed'])

display(Markdown("# Results - Table III and IV \n #### Comments about the unmatched values:"))
display(Markdown("` Features with conflicting values / Cases with conflicting feature values : `"))
display(Markdown("A/B == C -> can return NaN (0/0) or Inf (n/0) , if the expression is rearranged to A == C*B the outcomes for NaN will be 0 == 0"))
display(Markdown("but for Inf will be n == 0, in conclusion is plausible only to replace NaN by 0."))
display(Markdown("` Total Problem cases, DS'' / Total cases to remove : `"))
display(Markdown("It is notable that for DS''(M) they considered all duplicates except the first one... but for the Identical cases(G) all duplicates were counted."))
display(Markdown("Considering the definition of (M) is {Count of cases impacted by one or more of G to K} the correct value should be as shown on observed"))

df_result.style.apply(highlight_col, axis=None)


# Results - Table III and IV 
 #### Comments about the unmatched values:

` Features with conflicting values / Cases with conflicting feature values : `

A/B == C -> can return NaN (0/0) or Inf (n/0) , if the expression is rearranged to A == C*B the outcomes for NaN will be 0 == 0

but for Inf will be n == 0, in conclusion is plausible only to replace NaN by 0.

` Total Problem cases, DS'' / Total cases to remove : `

It is notable that for DS''(M) they considered all duplicates except the first one... but for the Identical cases(G) all duplicates were counted.

Considering the definition of (M) is {Count of cases impacted by one or more of G to K} the correct value should be as shown on observed

,Message,Expected,Observed
0,Identical features,0,0.000000
1,Constant features,0,0.000000
2,Features with missing values,0,0.000000
3,Features with conflicting values,2,9.000000
4,Features with implausible values,1,1.000000
5,Total Problem Features,3,10.000000
6,Identical cases,4621,4621.000000
7,Inconsistent cases,100,100.000000
8,Cases with missing values,0,0.000000
9,Cases with conflicting feature values,129,271.000000


## Clean

### Step 1 - remove cases with implausible values or conflict feature values    

In [391]:
df_DS1 = df.copy()
df_index_to_remove_1 = set(p_K_rows | p_J_rows)
df_DS1 = df_DS1.drop(df_index_to_remove_1)
df_DS1.shape,df.shape

((4415, 37), (5589, 37))

### Step 2 - remove identical cases   

In [392]:
df_index_to_remove_2 = p_G_rows_except_first - df_index_to_remove_1
df_DS2 = df_DS1.drop(df_index_to_remove_2)
df_DS2.shape

(1294, 37)

### Step 3 - remove inconsistent cases

In [393]:
df_index_to_remove_3 = p_H_rows - df_index_to_remove_2 - df_index_to_remove_1
df_DS3 = df_DS2.drop(df_index_to_remove_3)
df_DS3.shape

(1290, 37)

### Step 4 - remove cases missing values

In [394]:
df_index_to_remove_4 = p_I_rows - df_index_to_remove_3 - df_index_to_remove_2 - df_index_to_remove_1
df_DS4 = df_DS3.drop(df_index_to_remove_4)
df_DS4.shape

(1290, 37)

### Step 5 - remove constant features

In [395]:
df_columns_to_remove_1 = p_B_columns
df_DS5 = df_DS4.drop(df_columns_to_remove_1,axis='columns')
df_DS5.shape

(1290, 37)

### Step 6 - remove identical features

In [396]:
df_columns_to_remove_2 = p_A_columns - df_columns_to_remove_1    
df_DS6 = df_DS5.drop(df_columns_to_remove_2,axis='columns')
df_DS6.shape

(1290, 37)

## Save Dataset

In [397]:
df_clean = df_DS6.reset_index(drop=True)
print(df_clean.shape)
df_clean.head()

(1290, 37)


,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,DESIGN_COMPLEXITY,DESIGN_DENSITY,EDGE_COUNT,ESSENTIAL_COMPLEXITY,ESSENTIAL_DENSITY,LOC_EXECUTABLE,PARAMETER_COUNT,HALSTEAD_CONTENT,HALSTEAD_DIFFICULTY,HALSTEAD_EFFORT,HALSTEAD_ERROR_EST,HALSTEAD_LENGTH,HALSTEAD_LEVEL,HALSTEAD_PROG_TIME,HALSTEAD_VOLUME,MAINTENANCE_SEVERITY,MODIFIED_CONDITION_COUNT,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defects
0,1.0,4.0,7.0,24.0,0.0,1.0,0.13,0.0,0.0,1.0,1.0,6.0,1.0,0.0,1.0,0.0,17.88,7.43,986.77,0.04,34.0,0.13,54.82,132.83,1.0,0.0,0.0,7.0,0.03,13.0,21.0,7.0,8.0,34.0,96.88,8.0,False
1,1.0,1.0,11.0,3.0,0.0,1.0,0.08,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,42.62,7.81,2598.31,0.11,77.0,0.13,144.35,332.79,1.0,0.0,0.0,3.0,0.06,29.0,48.0,13.0,7.0,17.0,93.33,12.0,False
2,1.0,1.0,0.0,0.0,0.0,1.0,1.00,0.0,0.0,1.0,1.0,2.0,1.0,0.0,1.0,3.0,33.44,0.63,13.06,0.01,9.0,1.60,0.73,20.90,1.0,0.0,0.0,3.0,0.33,5.0,4.0,4.0,1.0,3.0,0.00,1.0,False
3,1.0,0.0,1.0,0.0,0.0,1.0,0.50,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,10.87,3.50,133.19,0.01,11.0,0.29,7.40,38.05,1.0,0.0,0.0,2.0,0.33,4.0,7.0,4.0,7.0,3.0,50.00,2.0,False
4,3.0,1.0,1.0,0.0,6.0,2.0,1.00,2.0,3.0,1.0,0.5,5.0,1.0,0.0,1.0,1.0,11.07,4.38,211.89,0.02,14.0,0.23,11.77,48.43,0.5,2.0,3.0,5.0,0.50,5.0,9.0,4.0,7.0,4.0,50.00,2.0,False


In [398]:
df_clean.to_pickle(f"{ROOT}/data/data.pkl")